# Build your own OpenAI Agent

With the [new OpenAI API](https://openai.com/blog/function-calling-and-other-api-updates) that supports function calling, it's never been easier to build your own agent!

In this notebook tutorial, we showcase how to write your own OpenAI agent in **under 50 lines of code**! It is minimal, yet feature complete (with ability to carry on a conversation and use tools).

## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

In [2]:
import json
from typing import Sequence, List

from llama_index.llms import OpenAI, ChatMessage
from llama_index.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

Let's define some very simple calculator tools for our agent.

In [3]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [4]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

## Agent Definition

Now, we define our agent that's capable of holding a conversation and calling tools in **under 50 lines of code**.

The meat of the agent logic is in the `chat` method. At a high-level, there are 3 steps:
1. Call OpenAI to decide which tool (if any) to call and with what arguments.
2. Call the tool with the arguments to obtain an output
3. Call OpenAI to synthesize a response from the conversation context and the tool output.

The `reset` method simply resets the conversation context, so we can start another conversation.

In [4]:
class YourOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: OpenAI = OpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role="user", content=message))
        functions = [
            tool.metadata.to_openai_function() for _, tool in self._tools.items()
        ]

        ai_message = self._llm.chat(chat_history, functions=functions).message
        chat_history.append(ai_message)

        function_call = ai_message.additional_kwargs.get("function_call", None)
        if function_call is not None:
            function_message = self._call_function(function_call)
            chat_history.append(function_message)
            ai_message = self._llm.chat(chat_history).message
            chat_history.append(ai_message)

        return ai_message.content

    def _call_function(self, function_call: dict) -> ChatMessage:
        tool = self._tools[function_call["name"]]
        output = tool(**json.loads(function_call["arguments"]))
        return ChatMessage(
            name=function_call["name"],
            content=str(output),
            role="function",
            additional_kwargs={"name": function_call["name"]},
        )

## Let's Try It Out!

In [5]:
agent = YourOpenAIAgent(tools=[multiply_tool, add_tool])

In [6]:
agent.chat("Hi")

'Hello! How can I assist you today?'

In [7]:
agent.chat("What is 2123 * 215123")

'The product of 2123 multiplied by 215123 is 456,706,129.'

## Our (Slightly Better) `OpenAIAgent` Implementation 

We provide a (slightly better) `OpenAIAgent` implementation in LlamaIndex, which you can directly use as follows.  

In comparison to the simplified version above:
* it implements the `BaseChatEngine` and `BaseQueryEngine` interface, so you can more seamlessly use it in the LlamaIndex framework. 
* it supports multiple function calls per conversation turn
* it supports streaming
* it supports async endpoints
* it supports callback and tracing

In [5]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

In [6]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

### Chat

In [7]:
response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405
(121 * 3) + 42 is equal to 405.


In [8]:
# inspect sources
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]


### Async Chat

In [9]:
response = await agent.achat("What is 121 * 3?")
print(str(response))

(121 * 3) + 42 is equal to 405.


### Streaming Chat
Here, every LLM response is returned as a generator. You can stream every incremental step, or only the last response.

In [10]:
response = agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363
121 * 2 is equal to 242.

Once upon a time, in a small village, there was a group of mice who lived happily in a cozy little burrow. The leader of the group was a wise and courageous mouse named Max. Max was known for his intelligence and his ability to solve problems.

One day, Max gathered all the mice together and announced that they were facing a shortage of food. The mice had been relying on the village's grain storage, but it was running low. Max knew that they needed to find a solution quickly to ensure the survival of their group.

Inspired by the number 242, Max came up with a brilliant plan. He organized the mice into teams and assigned each team a specific task. Some mice were tasked with scouting for new food sources, while others were responsible for gathering and storing food. Max himself took charge of coordinating the efforts and ensuring that everything ran smoothly.

### Async Streaming Chat

In [14]:
response = await agent.astream_chat(
    "What is 121 + 8? Once you have the answer, use that number to write a story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

121 + 8 is equal to 129.

Once upon a time, in a small village nestled in the countryside, there was a group of mice who lived in a cozy little burrow beneath a towering oak tree. The mice were known for their unity and their ability to overcome challenges together.

One sunny morning, as the mice were going about their daily activities, they stumbled upon a mysterious object hidden among the fallen leaves. It was a small, golden key with intricate engravings. Curiosity piqued, the mice decided to unlock the secret that lay before them.

The leader of the mouse group, a wise and adventurous mouse named Mia, took the key in her tiny paws and inserted it into a hidden lock they discovered nearby. With a gentle turn, a hidden door creaked open, revealing a hidden treasure room filled with delicious treats and treasures.

The mice couldn't believe their eyes! The room was filled with an abundance of cheese, nuts, and grains. It was a feast fit for a king! Overwhelmed with joy, the mice was

### Agent with Personality

You can specify a system prompt to give the agent additional instruction or personality.

In [13]:
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI
from llama_index.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [ ]:
response = agent.chat("Hi")
print(response)

In [ ]:
response = agent.chat("Tell me a story")
print(response)